# Method 1: Pure Collaborative-LightFM frequency dependent

In [72]:
# Let's start from the beggining
#Imports
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import os
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error 
import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from sklearn import preprocessing
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
data= pd.read_csv("150kGenres.csv")

In [74]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [75]:
#Lets see if two users have the same songs, we need to take out these
# to see the total ammount of songs in a dataset
data.shape

(185712, 4)

In [76]:



data.columns = data.columns.str.replace('"', '')
data.columns = data.columns.str.replace('name', '')
data.columns = data.columns.str.replace(' ', '')
data.columns


Index(['user_id', 'artist', 'track', 'genre'], dtype='object')

In [77]:
data.head()

,user_id,artist,track,genre
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,TBD
1,9cc0cfd4d7d7885102480dd99e7a90d6,Crowded House,Don't Dream It's Over,Rock 1980s
2,9cc0cfd4d7d7885102480dd99e7a90d6,Crowded House,Weather With You,Rock 1980s
3,9cc0cfd4d7d7885102480dd99e7a90d6,Pearl Jam,Alive,Rock 1990s
4,9cc0cfd4d7d7885102480dd99e7a90d6,Pearl Jam,Black,Rock 1990s


In [79]:
#Limiting results for it to be easier on Memory


df_playlist = data.groupby('artist').filter(lambda x : len(x)>=30)

df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') > 10]


In [80]:
df_playlist

,user_id,artist,track,genre
10,07f0fc3be95dcd878966b1f9572ff670,Zero 7,Destiny,Easy
11,07f0fc3be95dcd878966b1f9572ff670,Zero 7,Distractions,Jazz
12,07f0fc3be95dcd878966b1f9572ff670,Zero 7,Home,Dance 2000s
13,07f0fc3be95dcd878966b1f9572ff670,Johnny Cash,Hurt,TBD
14,07f0fc3be95dcd878966b1f9572ff670,Zero 7,I Have Seen,Jazz
...,...,...,...,...
185707,0a3e8220005a22ea58217bb2da98b639,Moby,Honey,Rock 1990s
185708,0a3e8220005a22ea58217bb2da98b639,Zero 7,I Have Seen,Jazz
185709,0a3e8220005a22ea58217bb2da98b639,Moby,If Things Were Perfect,Rock 1990s
185710,0a3e8220005a22ea58217bb2da98b639,Moby,Inside,Rock 1990s


In [81]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['user_id', 'artist','track','genre']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist','track','genre', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()


,user_id,artist,track,genre,freq
902,01ad7fc79498ab698b3f0e0f2c5c818e,MGMT,Electric Feel,Rock 2000s,11
948,01ad7fc79498ab698b3f0e0f2c5c818e,Nicki Minaj,Super Bass,CHR,10
714,01ad7fc79498ab698b3f0e0f2c5c818e,Dave Matthews Band,So Much to Say,Rock 1990s,10
716,01ad7fc79498ab698b3f0e0f2c5c818e,Dave Matthews Band,Tripping Billies,Rock 1990s,10
717,01ad7fc79498ab698b3f0e0f2c5c818e,Dave Matthews Band,Two Step,Rock 1990s,10


In [82]:
df_artist = pd.DataFrame(df_freq["track"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'track_id', 0:'track'})
df_artist.head()

,track_id,track
0,0,Electric Feel
1,1,Super Bass
2,2,So Much to Say
3,3,Tripping Billies
4,4,Two Step


In [83]:
df_freq  = pd.merge(df_freq , df_artist, how='inner', on='track')

In [84]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [85]:
# https://github.com/aayushmnit/cookbook/blob/master/recsys.py
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [86]:
# https://github.com/aayushmnit/cookbook/blob/master/recsys.py
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [112]:
# https://github.com/aayushmnit/cookbook/blob/master/recsys.py
def runMF(interactions, n_components=30, loss='warp', k=5, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    
    #uncommented for train test split
#     x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [88]:
# https://github.com/aayushmnit/cookbook/blob/master/recsys.py
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 1, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [104]:
interactions = create_interaction_matrix(df = df_freq, user_col = "user_id", item_col = 'track_id', rating_col = 'freq', norm= False, threshold = None)
interactions.head()

track_id,0,1,2,3,4,5,6,7,8,9,...,6790,6791,6792,6793,6794,6795,6796,6797,6798,6799
user_id,,,,,,,,,,,,,,,,,,,,,
001c4e5b73eca68ee9756bb0c7d2f855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002544ea04896444d79d3e4a7e073422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0098b965803a4c10723f8e216f9e0904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00abf74ef96b119cf0052a3d86fd4425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00b5474f01e21d9174c0b5c2b8e67c97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
interactions.shape

(1635, 6800)

In [91]:
user_dict = create_user_dict(interactions=interactions)

In [92]:
artists_dict = create_item_dict(df = df_artist, id_col = 'track_id', name_col = "track")
artists_dict

{0: 'Electric Feel',
 1: 'Super Bass',
 2: 'So Much to Say',
 3: 'Tripping Billies',
 4: 'Two Step',
 5: '1901',
 6: 'One Headlight',
 7: "I Won't Give Up",
 8: 'Royals',
 9: '#41',
 10: 'Electrolite',
 11: 'Gamma Ray',
 12: 'Let Her Go',
 13: 'Too Much',
 14: 'Say Goodbye',
 15: 'Proudest Monkey',
 16: 'Let You Down',
 17: 'There She Goes',
 18: 'Adorn',
 19: 'Tongue Tied',
 20: 'Yellow',
 21: 'Chicken Fried',
 22: 'Pompeii',
 23: 'Chocolate',
 24: 'Fix You',
 25: 'Take Me Out',
 26: 'Mardy Bum',
 27: 'A Milli',
 28: 'Everlong',
 29: 'A Certain Romance',
 30: 'San Francisco',
 31: 'Crush',
 32: 'Lullaby',
 33: "It's Time",
 34: 'Crash Into Me',
 35: 'Dancing On My Own',
 36: 'Superstar',
 37: 'Bulletproof',
 38: 'Lego House',
 39: 'Viva La Vida',
 40: 'Good',
 41: 'Real World',
 42: 'I Feel It All',
 43: 'Chasing Cars',
 44: 'Glycerine',
 45: "Don't Stop Me Now",
 46: 'Tonight Is The Night',
 47: 'Kids',
 48: 'Crooked Teeth',
 49: 'Violet Hill',
 50: 'The Ghosts Of Beverly Drive',
 51

In [93]:
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)


In [113]:
%time
model = runMF(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 k = 5,
                 epoch = 30,
                 n_jobs = 4)

CPU times: total: 0 ns
Wall time: 0 ns


In [114]:
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)

Train AUC: 0.9716839


In [115]:


test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)



Test AUC: 0.97279567


In [116]:


train_precision = precision_at_k(model, train, k=5).mean()
test_precision = precision_at_k(model, test, k=5, train_interactions=train).mean()



In [117]:
print('Train Precision %.2f, Test Precision %.2f.' % (train_precision, test_precision))

Train Precision 0.40, Test Precision 0.22.


In [99]:
artists_dict

{0: 'Electric Feel',
 1: 'Super Bass',
 2: 'So Much to Say',
 3: 'Tripping Billies',
 4: 'Two Step',
 5: '1901',
 6: 'One Headlight',
 7: "I Won't Give Up",
 8: 'Royals',
 9: '#41',
 10: 'Electrolite',
 11: 'Gamma Ray',
 12: 'Let Her Go',
 13: 'Too Much',
 14: 'Say Goodbye',
 15: 'Proudest Monkey',
 16: 'Let You Down',
 17: 'There She Goes',
 18: 'Adorn',
 19: 'Tongue Tied',
 20: 'Yellow',
 21: 'Chicken Fried',
 22: 'Pompeii',
 23: 'Chocolate',
 24: 'Fix You',
 25: 'Take Me Out',
 26: 'Mardy Bum',
 27: 'A Milli',
 28: 'Everlong',
 29: 'A Certain Romance',
 30: 'San Francisco',
 31: 'Crush',
 32: 'Lullaby',
 33: "It's Time",
 34: 'Crash Into Me',
 35: 'Dancing On My Own',
 36: 'Superstar',
 37: 'Bulletproof',
 38: 'Lego House',
 39: 'Viva La Vida',
 40: 'Good',
 41: 'Real World',
 42: 'I Feel It All',
 43: 'Chasing Cars',
 44: 'Glycerine',
 45: "Don't Stop Me Now",
 46: 'Tonight Is The Night',
 47: 'Kids',
 48: 'Crooked Teeth',
 49: 'Violet Hill',
 50: 'The Ghosts Of Beverly Drive',
 51

In [61]:
user_dict

{'001c4e5b73eca68ee9756bb0c7d2f855': 0,
 '002544ea04896444d79d3e4a7e073422': 1,
 '0098b965803a4c10723f8e216f9e0904': 2,
 '00abf74ef96b119cf0052a3d86fd4425': 3,
 '00b5474f01e21d9174c0b5c2b8e67c97': 4,
 '00f38a9f7e21c349643b32489bda6ff3': 5,
 '01032d628bd015d468a9849bc02b9260': 6,
 '013563dd7de2dc6431a3517d5ccdbef0': 7,
 '016a8eb7c9e5f511bd3749552ab19e56': 8,
 '01ad7fc79498ab698b3f0e0f2c5c818e': 9,
 '01ae27762c085ae2d6f202325debe054': 10,
 '01d0ffb3f21ebe904adc4c03544055b4': 11,
 '01e55d8ea1461b270889e4d205205eb9': 12,
 '0255f043d69f1bedc070f72e94b01c05': 13,
 '02ab8fee3a456231efdacd1947bd5890': 14,
 '02e7e8b832c68c0e38e6af926a225421': 15,
 '02ecff5b6c216a6bb5b5dd9c13de71f3': 16,
 '0308284ac2d49b8692103f04b6ac741b': 17,
 '031d69a7c94fdedb5e8648916af3a4e4': 18,
 '0326d7137b68e3ce019c3a5ba9b42d16': 19,
 '0341b9badfb5c065785174c1cb52ed41': 20,
 '03528e2dca6adaf0879c143f0b175a9a': 21,
 '03a3ff0b6acd6fe16474887e3287025f': 22,
 '03b3c34198301598b0908862a4f86138': 23,
 '03bf70929522d0fa80ddbfa6

In [100]:
df_freq.value_counts("user_id")

user_id
6a84fc1511d2a79f951d7d22a7872861    1111
61baddf7207fea410abdc56e680fa869    1019
e3833c156aef7b25ba9d8837341a99bb     875
8b114b8f5d94812867c29e92306f42f7     834
b38804ddef21178dd8a94dc897a2ea69     678
                                    ... 
e5d0573bd749b427205245349a97981d      11
c833d87449d3d0e788c3a373183d701f      11
8ea9edf1097f0cda0bb051721be84fd4      11
44bc467769bcf061a038f10fda5d69f5      11
052d225d5d16e1b3c3932b20d200d29e      11
Length: 1635, dtype: int64

In [120]:
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = '052d225d5d16e1b3c3932b20d200d29e', 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 1,
                                      show = True)

Known Likes:
1- Caravan
2- The Pot
3- Demons
4- Losing My Religion
5- Scar Tissue
6- Semi-Charmed Life
7- Luka
8- Two Princes
9- Get Lucky
10- Push
11- Counting Stars

 Recommended Items:
1- Smells Like Teen Spirit
